In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data 
from torch.autograd import Variable

In [2]:
movies = pd.read_csv("./data/ml-1m/movies.dat", sep = "::", header=None, engine='python', encoding='latin-1')

In [3]:
users = pd.read_csv("./data/ml-1m/users.dat", sep = "::", header=None, engine='python', encoding='latin-1')

In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
ratings = pd.read_csv("./data/ml-1m/ratings.dat", sep = "::", header=None, engine='python', encoding='latin-1')

In [7]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
#Prepare train and test data - using k-fld cross valiation
train_set = pd.read_csv("./data/ml-100k/u1.base", delimiter='\t')

In [9]:
print(len(train_set))

79999


In [10]:
train_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [11]:
train_set = np.array(train_set, dtype='int')

In [12]:
test_set = pd.read_csv("./data/ml-100k/u1.test", delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [13]:
print(len(test_set))

19999


In [14]:
#total user and movies
nb_users = int(max(max(train_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(train_set[:,1]), max(test_set[:,1])))

In [15]:
print(nb_users)
print(nb_movies)

943
1682


In [16]:
#Convert data such that users are in rows and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [17]:
train_set = convert(train_set)
test_set = convert(test_set)

In [18]:
#convert data into torch tensors
train_set = torch.FloatTensor(train_set)
test_set = torch.FloatTensor(test_set)

In [19]:
#Convert rating into binary 0 not liked and 1 liked
train_set[train_set == 0] = -1
train_set[train_set == 1] = 0
train_set[train_set == 2] = 0
train_set[train_set == 3] = 1
train_set[train_set == 4] = 1
train_set[train_set == 5] = 1

In [20]:
test_set[train_set == 0] = -1
test_set[train_set == 1] = 0
test_set[train_set == 2] = 0
test_set[train_set == 3] = 1
test_set[train_set == 4] = 1
test_set[train_set == 5] = 1

In [21]:
#Create a newral network architecture
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.b1 = torch.randn(1, nh) # 1 corresponds to batch
        self.b2 = torch.randn(1, nv)
        
    def sample_hidden(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.b1.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)  #sampling hidden neuron using bernoulli sampling
    
    def sample_visible(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b2.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)  #sampling visiable neurons from hidden layers
    
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b2 += torch.sum((v0 - vk), 0)
        self.b1 += torch.sum((ph0 - phk), 0)

In [22]:
nv = len(train_set[0]) # same as nb_movies
nh = 100 # no. of features that we want to learn about the problem, ex. genre, director, actor etc. This is hyper parameter
batch_size = 100 # Again a ahyper paarmeter
rbm = RBM(nv, nh)

In [23]:
#Train the rbm
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    counter = 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = train_set[id_user:id_user+batch_size]
        v0 = train_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_hidden(v0)
        for k in range(10):
            _,hk = rbm.sample_hidden(vk)
            _,vk = rbm.sample_visible(hk)
            vk[vk<0] = v0[v0<0]
        phk,_ = rbm.sample_hidden(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        counter += 1.
    print('epoch: '+ str(epoch) + ' Train Loss: ' + str(train_loss/counter))

epoch: 1 Train Loss: 0.787671358078
epoch: 2 Train Loss: 0.825150126953
epoch: 3 Train Loss: 0.825150126953
epoch: 4 Train Loss: 0.825150126953
epoch: 5 Train Loss: 0.825150126953
epoch: 6 Train Loss: 0.825150126953
epoch: 7 Train Loss: 0.825150126953
epoch: 8 Train Loss: 0.825150126953
epoch: 9 Train Loss: 0.825150126953
epoch: 10 Train Loss: 0.825150126953


In [26]:
#Testing the RBM

test_loss = 0
counter = 0.0
for id_user in range(nb_users):
    v = train_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt >= 0]) > 0:
        _,h = rbm.sample_hidden(v)
        _,v = rbm.sample_visible(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    counter += 1.
print('Test Loss: ' + str(test_loss/counter))

Test Loss: 0.0451985672779
